In [4]:
import numpy as np
class MNISTLoader():
  def __init__(self):
    mnist = tf.keras.datasets.mnist
    (self.train_data,self.trainn_label),(self.test_data,self.test_label) = mnist.load_data()
    #mnist = tf.keras.datasets.mnist
    #(training_images, training_labels), (test_images, test_labels) = mnist.load_data()

    self.train_data = np.expand_dims(self.train_data.astype(np.float32) / 255.0,axis = -1)
    self.test_data = np.expand_dims(self.test_data.astype(np.float32) / 255.0,axis = -1)
    self.trainn_label = self.trainn_label.astype(np.int32)
    self.test_label = self.test_label.astype(np.int32)

    self.num_train_data, self.num_test_data = self.train_data.shape[0], self.test_data.shape[0]

  def get_batch(self, batch_size):
    index = np.random.randint(0, np.shape(self.train_data)[0], batch_size)

    return self.train_data[index,:], self.trainn_label[index]

import numpy as np
class MNISTLoader():
  def __init__(self):
    mnist = tf.keras.datasets.mnist
    (self.train_data,self.trainn_label),(self.test_data,self.test_label) = mnist.load_data()
    #mnist = tf.keras.datasets.mnist
    #(training_images, training_labels), (test_images, test_labels) = mnist.load_data()

    self.train_data = np.expand_dims(self.train_data.astype(np.float32) / 255.0,axis = -1)
    self.test_data = np.expand_dims(self.test_data.astype(np.float32) / 255.0,axis = -1)
    self.trainn_label = self.trainn_label.astype(np.int32)
    self.test_label = self.test_label.astype(np.int32)

    self.num_train_data, self.num_test_data = self.train_data.shape[0], self.test_data.shape[0]

  def get_batch(self, batch_size):
    index = np.random.randint(0, np.shape(self.train_data)[0], batch_size)

    return self.train_data[index,:], self.trainn_label[index]

In [7]:
import tensorflow as tf
class CNN(tf.keras.Model):
  def __init__(self):
    super().__init__()

    self.conv1 = tf.keras.layers.Conv2D(filters=32,kernel_size=[5,5],padding='same',activation=tf.nn.relu)
    self.pool1 = tf.keras.layers.MaxPool2D(pool_size=[2,2],strides=2)
    self.conv2 = tf.keras.layers.Conv2D(filters=64,kernel_size=[5,5],padding='same',activation=tf.nn.relu)
    self.pool2 = tf.keras.layers.MaxPool2D(pool_size=[2,2],strides=2)
    self.flatten  = tf.keras.layers.Reshape(target_shape=(7*7*64,))
    self.dense1 = tf.keras.layers.Dense(units=1024, activation=tf.nn.relu)
    self.dense2 = tf.keras.layers.Dense(units=10)

  def call(self, inputs):

      X = self.conv1(inputs)
      X = self.pool1(X)
      X = self.conv2(X)
      X = self.pool2(X)
      X = self.flatten(X)
      X = self.dense1(X)
      X = self.dense2(X)

      output = tf.nn.softmax(X)

      return output

In [ ]:
from keras.engine.training import optimizer
#參數
num_epochs = 5
batch_size = 50
learning_rate = 0.001

#模型部分
model = CNN()
data_loader = MNISTLoader()
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

#運行模型
num_batches = int(data_loader.num_train_data // batch_size*num_epochs)

for batch_index in range(num_batches):
  X, y = data_loader.get_batch(batch_size)
  with tf.GradientTape() as tape:
    y_pred = model(X)
    loss = tf.keras.losses.sparse_categorical_crossentropy(y_true=y, y_pred=y_pred)
    loss = tf.reduce_mean(loss)

    print("batch %d loss %f"%(batch_index, loss.numpy()))

  grads = tape.gradient(loss, model.variables)
  optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))

11490434/11490434 [==============================] - 0s 0us/step
batch 0 loss 2.318264
batch 1 loss 2.268350
batch 2 loss 2.122263
batch 3 loss 1.979640
batch 4 loss 1.896778
batch 5 loss 1.645711
batch 6 loss 1.479392
batch 7 loss 1.303233
batch 8 loss 0.874884
batch 9 loss 1.275907
batch 10 loss 1.146882
batch 11 loss 0.861475
batch 12 loss 1.050670
batch 13 loss 0.692612
batch 14 loss 0.831446
batch 15 loss 0.760720
batch 16 loss 0.806661
batch 17 loss 0.528031
batch 18 loss 0.510168
batch 19 loss 0.379103
batch 20 loss 0.556742
batch 21 loss 0.809226
batch 22 loss 0.611753
batch 23 loss 0.426849
batch 24 loss 0.478830
batch 25 loss 0.416351
batch 26 loss 0.287383
batch 27 loss 0.319458
batch 28 loss 0.635495
batch 29 loss 0.446355
batch 30 loss 0.291160
batch 31 loss 0.540856
batch 32 loss 0.216799
batch 33 loss 0.483014
batch 34 loss 0.334234
batch 35 loss 0.280492
batch 36 loss 0.084890
batch 37 loss 0.226191
batch 38 loss 0.524950
batch 39 loss 0.382446
batch 40 loss 0.479681
ba

In [ ]:
from keras.metrics.accuracy_metrics import sparse_categorical_accuracy
from tensorflow.python.ops.batch_ops import batch
spase_categorical_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
num_batches = int(data_loader.num_test_data // batch_size)

for batch_index in range(num_batches):
  start_index, end_index = batch_index*batch_size, (batch_index + 1)*batch_size
  y_pred = model.predict(data_loader.test_data[start_index:end_index])


  #sparse_categorical_accuracy.update_state(y_true = data_loader.test_label[start_index:end_index],y_pred = y_pred)

  # 实例化度量对象
  sparse_categorical_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

  # 调用update_state方法来更新度量对象的内部状态
  sparse_categorical_accuracy.update_state(y_true = data_loader.test_label[start_index:end_index],y_pred = y_pred)

  print("test accuracy: %f" % sparse_categorical_accuracy.result())